In [79]:
import numpy as np
import pandas as pd
import os,sys

In [80]:
path_data = os.path.join(os.getcwd(),os.pardir,os.pardir,'data')
data_list = os.listdir(path_data)
print(data_list)

['bleaching', 'fishing_activity', 'mmsi-daily-csvs-10-v2-2012', 'mmsi-daily-csvs-10-v2-2013', 'mmsi-daily-csvs-10-v2-2014', 'mmsi-daily-csvs-10-v2-2015', 'mmsi-daily-csvs-10-v2-2016', 'mmsi-daily-csvs-10-v2-2017', 'mmsi-daily-csvs-10-v2-2018', 'mmsi-daily-csvs-10-v2-2019', 'protected_area']


In [81]:
year = 2012
filename = 'mmsi-daily-csvs-10-v2-' + str(year)
fishing_data_path = os.path.join(path_data,filename)
# encoding='ISO-8859-1' for read_csv

In [82]:
from datetime import datetime, timedelta
start_date = datetime(year, 1, 1)
end_date = datetime(year, 12, 31)
delta = timedelta(days=1)

aggregated_data = pd.DataFrame()
current_date = start_date
while current_date <= end_date:
    file_name = current_date.strftime('%Y-%m-%d') + '.csv'
    file_path = os.path.join(fishing_data_path, file_name)
    
    if os.path.exists(file_path):
        daily_data = pd.read_csv(file_path)
        daily_data = daily_data[['cell_ll_lat', 'cell_ll_lon', 'fishing_hours']]
        daily_data = daily_data[daily_data['fishing_hours'] > 0]
        
        daily_data = daily_data[
            ((daily_data['cell_ll_lat'] >= 0) & (daily_data['cell_ll_lat'] <= 40) &
             (daily_data['cell_ll_lon'] >= 100) & (daily_data['cell_ll_lon'] <= 140)) |
            ((daily_data['cell_ll_lat'] >= 0) & (daily_data['cell_ll_lat'] <= 40) &
             (daily_data['cell_ll_lon'] >= -100) & (daily_data['cell_ll_lon'] <= -60))
        ]

        if aggregated_data.empty:
            aggregated_data = daily_data
        else:
            aggregated_data = pd.concat([aggregated_data, daily_data])

    current_date += delta

aggregated_data = aggregated_data.groupby(['cell_ll_lat', 'cell_ll_lon']).sum().reset_index()

In [ ]:
# import pandas as pd
# from datetime import datetime, timedelta

# # Define the bin size for latitude and longitude
# lat_bin_size = 2
# lon_bin_size = 2

# def calculate_bin(value, bin_size):
#     bin_center = ((value + bin_size/2) // bin_size) * bin_size
#     return bin_center

# start_date = datetime(year, 1, 1)
# end_date = datetime(year, 12, 31)
# delta = timedelta(days=1)

# # Initialize the aggregated data
# aggregated_data = pd.DataFrame()

# current_date = start_date
# while current_date <= end_date:
#     file_name = current_date.strftime('%Y-%m-%d') + '.csv'
#     file_path = os.path.join(fishing_data_path, file_name)
    
#     if os.path.exists(file_path):
#         daily_data = pd.read_csv(file_path)
#         daily_data = daily_data[['cell_ll_lat', 'cell_ll_lon', 'fishing_hours']]
#         daily_data = daily_data[daily_data['fishing_hours'] > 0]

#     # Bin the data by rounding down to the nearest multiple of 5 degrees
#     # daily_data['lat_bin'] = (daily_data['cell_ll_lat'] // lat_bin_size) * lat_bin_size
#     # daily_data['lon_bin'] = (daily_data['cell_ll_lon'] // lon_bin_size) * lon_bin_size
#     daily_data['lat_bin'] = daily_data['cell_ll_lat'].apply(calculate_bin, bin_size=lat_bin_size)
#     daily_data['lon_bin'] = daily_data['cell_ll_lon'].apply(calculate_bin, bin_size=lon_bin_size)
    
#     # Group by the binned lat/lon and sum the 'fishing_hours', then compute the mean
#     daily_grouped = daily_data.groupby(['lat_bin', 'lon_bin']).agg({'fishing_hours': 'mean'}).reset_index()
    
#     # Concatenate the grouped data to the aggregated data
#     aggregated_data = pd.concat([aggregated_data, daily_grouped])
    
#     # Increment the date by one day
#     current_date += delta

# # Final aggregation across all days, summing up 'fishing_hours' for each bin
# aggregated_data = aggregated_data.groupby(['lat_bin', 'lon_bin']).sum().reset_index()


In [ ]:
save_folder = os.path.join(os.getcwd(),os.pardir,os.pardir,'data','fishing_activity')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

save_file = os.path.join(save_folder,'fishing_activity_' + str(year) + '.csv')
aggregated_data.to_csv(save_file, index=False)